# Homework 6 - Linked List Labyrinth

<b>Due Date</b>: 11/2/2023 at 11:59pm<br>
<b>Points</b> - 1000 points<br>

><b>Qualitative Feedback</b>: The objective of this assignment is to give you practice with structs, linked lists, pointers, and memory allocation. You will demonstrate your ability to implement:
><ul>
>    <li>Advanced C Structs</li>
>    <li>Command Line inputs</li>
>    <li>Recursion</li>
>    <li>Depth-First Search</li>
     <li>Developing Your Own Test Cases</li>
></ul>

## Setup Steps

<b>1.</b> Open Docker Desktop. Go to the cse20133-user Container you created. Press the Blue Triangle to start the Container.

<b>2.</b> Open VSCode, and on the left, select the blue button in the bottom left of VSCode. At the top, select the pull-down choice “Attach to running container” and select your CSE 20133 course container.

<b>3.</b> Go into your Git Folder:

> Recall that @USERNAME is the unique username you created when you created your GitHub account. You will see your user name in the VS Code Docker

    cd cse20133-user/cse20133-@USERNAME

Create the folder:

    mkdir homework06
    cd homework06

4 - You will need to create the following files in your <code>homework4</code> folder:
<ul>
    <li><code>homework06.c</code></li>
    <li><code>labyrinth.c</code></li>
    <li><code>maze_generator.c</code></li>
    <li><code>Makefile</code></li>
</ul>

You will be provided the header files for this assignment, <code>labyrinth.h</code> and <code>maze_generator.h</code>. You can obtain them by performing these commands in your <code>homework06</code> folder:

    wget https://raw.githubusercontent.com/mmorri22/cse20133/main/homeworks/homework6/labyrinth.h



When your assignment is completed, you will need to be able to run the command <code>make homework05</code>, and produce the output and compiler flags as shown below. You must also pass <code>valgrind</code> since we are performing dynamic memory allocation.

<code><font size="2" color="white" style="background-color:black;">> make homework05</font></code><br>
<code><font size="2" color="white" style="background-color:black;">gcc -std=c2x -O2 -g -Wall -Wextra -Wconversion -Wshadow -Werror -c rotations.c</font></code><br>
<code><font size="2" color="white" style="background-color:black;">gcc -std=c2x -O2 -g -Wall -Wextra -Wconversion -Wshadow -Werror -c homework05.c</font></code><br>
<code><font size="2" color="white" style="background-color:black;">gcc -std=c2x -O2 -g -Wall -Wextra -Wconversion -Wshadow -Werror -o homework05 homework05.o rotations.o -lm</font></code><br>
<code><font size="2" color="white" style="background-color:black;">valgrind --leak-check=full ./homework05</font></code></font><br>